In [ ]:
from autumn.tools.project import get_project
from autumn.settings import Region
from matplotlib import pyplot
from autumn.tools.plots.utils import REF_DATE
from autumn.tools.calibration.targets import get_target_series
import pandas as pd
from autumn.tools.utils.pretty import pretty_print

import timeit

In [ ]:
project = get_project("covid_19", "sri_lanka")

In [ ]:
params_baseline = project.param_set.baseline

In [ ]:
model = project.run_baseline_model(params_baseline)

In [ ]:
model.compartments

In [ ]:
set([i.source.strata["history"] for i in model._flows if i.name=="recovery"])

In [ ]:
set([i.source.name for i in model._flows if i.name=="recovery"])

In [ ]:
derived_df = model.get_derived_outputs_df()

In [ ]:
def plot_derived_outputs(requested_outputs):

    # Preparing
    fig = pyplot.figure(figsize=(14, 8))
    for i_out, output in enumerate(requested_outputs):

        # Plotting
        axis = fig.add_subplot(2, 2, i_out + 1)
        axis.plot(derived_df[output])

        try:
            target_dates, target_values = get_target_series(project.calibration.targets, REF_DATE, output)
            axis.scatter(target_dates, target_values, c="k")
        except:
            print(f"targets not available for {output}")

        # Finishing up
        axis.set_title(output)
        axis.set_ylim(bottom=0.)
        if "prop_" in output:
            axis.set_ylim(top=1.)
        pyplot.setp(axis.get_xticklabels(), rotation=45)
    fig.tight_layout()

# vaccine coverage

In [ ]:
[col for col in derived_df.columns if "proportion_" in col]

In [ ]:
fig = pyplot.figure(figsize=(14, 8))
axis = fig.add_subplot(1, 1, 1)

at_least_one_dose = derived_df["at_least_one_dose_prop"]

axis.fill_between(
    derived_df.index,
    [0.] * len(derived_df),
    derived_df["proportion_waned"],
    label="waned immunity"
)
axis.fill_between(
    derived_df.index,
    derived_df["proportion_waned"],
    derived_df["proportion_part_waned"] + derived_df["proportion_waned"],
    label="partially waned immunity"
)
axis.fill_between(
    derived_df.index,
    derived_df["proportion_waned"] + derived_df["proportion_part_waned"],
    derived_df["proportion_fully_vaccinated"] +derived_df["proportion_waned"] + derived_df["proportion_part_waned"],
    label="fully vaccinated"
)
axis.fill_between(
    derived_df.index,
    derived_df["proportion_fully_vaccinated"] + derived_df["proportion_waned"] + derived_df["proportion_part_waned"],
    derived_df["proportion_one_dose"] + derived_df["proportion_fully_vaccinated"] +derived_df["proportion_waned"] + derived_df["proportion_part_waned"],
    label="one dose only"
)
axis.fill_between(
    derived_df.index,
    derived_df["proportion_one_dose"] + derived_df["proportion_fully_vaccinated"] + derived_df["proportion_waned"] + derived_df["proportion_part_waned"],
    [1.] * len(derived_df),
    label="unvaccinated"
)
axis.legend()